# Distribution du calcul

## Contexte

L'objectif du projet était l'application de la SVD sur le dataset récupéré. Mais, il n'a pas été possible d'appliquer la SVD à notre matrice. La matrice (films,utilisateurs) étant de taille trop importante, il est impossible de la charger en mémoire, sur un seul ordinateur.

Ainsi, nous avons essayer de distribuer la matrice dans un cluster, puis distribuer le calcul.

Nous avons donc choisi d'utiliser *Dask* dans un premier temps puis nous avons essaye avec *Dispy*.

## Dask

Dask permet de distribuer le calcul sur différents coeurs d'une même machine ou sur différents noeuds d'un cluster.

#### Sur une seule machine :

In [21]:
from dask.distributed import Client

Dans un premier temps, nous avons essayé le calcul sur un même ordinateur et voir la répartition entre les différents workers.
Pour cela, il suffit d'instancier un client Dask.

In [22]:
client = Client() # mise en place du cluster sur l'ordinateur

In [23]:
client

Client Scheduler: tcp://127.0.0.1:52927 Dashboard: http://127.0.0.1:52928/status,Cluster Workers: 4 Cores: 4 Memory: 17.18 GB


Dask fournit également un dashboard afin de mieux visualiser la comportement du système.
Par défaut, le client instancie autant de workers que de cores disponibles.

Nous avons repris l'exemple fournit par Dask.

In [24]:
def square(x):
    #print("Process {} working square function".format(os.getpid()))
    result = x ** 2
    #print("Process {} done processing square function".format(os.getpid()))
    return result

In [25]:
def neg(x):
    return -x

In [26]:
A = client.map(square, range(10000)) # Map a function on a sequence of arguments
B = client.map(neg,A)
total = client.submit(sum, B) # Submit a function application to the scheduler

In [27]:
total

<Future: status: pending, key: sum-acc9d4bf554402c0b81eb9a548b097b2>

![Dash dashboard](/Users/alessandro/Desktop/pfe/recommender-systems/images/dask_dashboard.png)

On distingue donc bien les différentes tâches qui se sont exécutées sur les différents workers.

#### Au sein d'un cluster :

Dans un second temps, nous avons voulu essayer de reproduire cela sur deux noeuds d'un cluster. 
La mise en place du cluster s'est bien déroulé. Chacun des workers était visible par le scheduler, mais nous n'avons jamais réussi distribuer le calcul dans le cluster.
Nous n'avions qu'un seul de nos noeuds qui travaillait.

## Dispy

Dispy permet également de distribuer les tâches d'un programme sur différents noeuds du cluster. 
C'est ce que nous avons réussi à faire pour un programme simple.

Dans un premier temps, nous avons réussi à distribuer nos tâches sur deux noeuds. La répartition des tâches se faisait en fonction du nombre de coeurs de CPU. 

Par exemple, pour 14 tâches à exécuter, 8 étaient effectuées sur un noeuds de 8 coeurs, et 6 sur un autre noeuds disposant lui aussi 8 coeurs.





#### Essai

On commence par lancer un dispynode sur chacun des nodes.

![Lancement du dispynode](/Users/alessandro/Desktop/pfe/recommender-systems/images/dispynode_terminal.png)

Puis, on ouvre un autre terminal dans lequel on lance le programme python.
Ce programme est écrit de sorte à distribuer les jobs ou tâches sur les différents noeuds du cluster.

![Algorithme](/Users/alessandro/Desktop/pfe/recommender-systems/images/dispy_algo.png)

On observe ainsi les logs du programme.
![Computation](/Users/alessandro/Desktop/pfe/recommender-systems/images/dispy_script_python.png)

#### Conclusion

Malheureusement, nous n'avons pas pu distribuer notre programme de SVD sur différents noeuds. 
En effet, *Dispy* distribue les tâches mais dans notre cas, nous ne pouvons pas séparer nos calculs en différentes tâches. Il est nécessaire pour nous d'appliquer le calcul à l'ensemble de notre dataset.

Nous en avons donc conclu que notre programme n'était finalement pas distribuable.